In [372]:
# Standard library imports
from datetime import datetime
import os

# Third-party library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm  # Visualize loop progress

# Set pandas display options for better data frame visualization
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Segreti
from dotenv import load_dotenv
load_dotenv()

# API
import requests
import json

| Gap-Up Screener |
|--|
TradingView Data (import)

In [373]:
# today's date string for file naming automation
# today_date_str = datetime.now().strftime("_%Y-%m-%d.csv")
today_date_str = '_2024-03-07.csv'

print(today_date_str)

_2024-03-07.csv


In [374]:
# concat base file name with today's date string
filename = f"trading_view_raw_data/tv_screen_gap-up{today_date_str}"
# read in trading view raw data
trading_view_raw_df = pd.read_csv(filename)

print(f'{len(trading_view_raw_df.index)} U.S. Stocks \n$(pre-market change) > $0.00 USD')
trading_view_raw_df.head(5)

1645 U.S. Stocks 
$(pre-market change) > $0.00 USD


,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower"
0,MSFT,Microsoft Corporation,NASDAQ,3029388743942.5767,USD,407.7000,USD,401.40,USD,4.14,USD,1.029620,-0.171603,7318938817.36,6038168,72897450,153732,18317632.3,22653584.93333334,24695687.58888889,1.521480,1.802997,1.456598,406.050000,36.87356987,Technology Services,1.33390343,0.9529795,1.0291758,0.8906199,0.006283,0.000648,0.102547,0.069518,0.511495,1.35116077,0.72525565,0.29577401,0.69349908,0.17078787,420.820,USD,420.82,USD,420.82,USD,420.82,USD,420.82000,USD,407.7700,USD,407.7840,USD,407.7840,USD,408.2100,USD,408.2100,USD,408.2100,USD,408.2100,USD,417.35,USD,417.35,USD,958891.40271493,USD,407.44680,407.458012,407.66144,406.709715,408.077949,407.557865,407.037781,408.834448,405.770310,402.706172
1,NVDA,NVIDIA Corporation,NASDAQ,2294911000000,USD,917.9644,USD,889.17,USD,14.55,USD,1.640361,0.244645,2399492500,24983933,196861538,963981,50731960.4,52039927.09999999,45684760.52222235,3.669561,3.465403,3.660522,911.288167,76.90398358,Electronic Technology,1.55029011,1.8023118,2.3419523,1.7245569,0.003040,0.000376,0.169068,0.108818,0.714603,1.8226155,0.93532542,0.43920408,0.87974948,0.24321959,919.880,USD,919.88,USD,919.88,USD,919.88,USD,919.88000,USD,917.7846,USD,918.6000,USD,918.6000,USD,919.8800,USD,919.8800,USD,919.8800,USD,919.8800,USD,919.88,USD,919.88,USD,2058175.67567568,USD,917.04892,917.041488,917.50708,914.952654,919.480428,917.509580,915.538732,922.645829,911.518965,900.392101
2,AMZN,"Amazon.com, Inc.",NASDAQ,1826433893442.8735,USD,175.8320,USD,172.50,USD,1.32,USD,0.760763,-0.582099,9227833763.2,10147300,116848089,216491,36604066.00000006,47647839.56666667,48387894.11111113,1.818962,1.727769,1.783302,175.407300,60.84994463,Retail Trade,1.18742275,1.1343786,1.3817025,1.1703976,0.002072,0.000255,0.231699,0.153234,0.740425,1.57087472,0.68843918,0.24672143,0.54233525,0.13228522,180.140,USD,180.14,USD,180.14,USD,180.14,USD,188.65391,USD,175.8100,USD,175.9699,USD,175.9699,USD,176.6699,USD,176.6699,USD,176.6699,USD,176.6699,USD,180.14,USD,180.14,USD,376908.19672131,USD,175.75702,175.806900,176.03200,175.929615,176.618810,176.080100,175.541390,177.031126,175.433745,173.836364
3,GOOGL,Alphabet Inc.,NASDAQ,1667862074561.7778,USD,133.6100

In [375]:
def categorize_market_cap(df):
    # convert 'Market capitalization' to numeric, coercing errors to NaN
    df['Market capitalization'] = pd.to_numeric(df['Market capitalization'], errors='coerce')
    # define conditions for market cap categories
    conditions = [
        # Titans
        (df['Market capitalization'] >= 200000000000),  # 200 billion and above
        #Large Cap
        (df['Market capitalization'] >= 10000000000) & (df['Market capitalization'] < 200000000000),  # 10 billion to 200 billion
        # Midlers
        (df['Market capitalization'] >= 2000000000) & (df['Market capitalization'] < 10000000000),  # 2 billion to 10 billion
        # Small Cap
        (df['Market capitalization'] >= 300000000) & (df['Market capitalization'] < 2000000000),  # 300 million to 2 billion
        # Micro Cap
        (df['Market capitalization'] > 50000000) & (df['Market capitalization'] < 300000000),  # 300 million and below (excluding 50 million and below)
        # Shrimp
        (df['Market capitalization'] <= 50000000)  # 50 million and below
    ]
    categories = ['Titans', 'Large caps', 'Midlers', 'Small caps', 'Micro caps', 'Shrimp']
    # use np.select to assign categories based on conditions
    df['marketCapType'] = np.select(conditions, categories, default='Undefined')
    
    return df


In [376]:
# EXECUTE
trading_view_raw_df = categorize_market_cap(trading_view_raw_df)
trading_view_raw_df.head(5)

,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
0,MSFT,Microsoft Corporation,NASDAQ,3.029389e+12,USD,407.7000,USD,401.40,USD,4.14,USD,1.029620,-0.171603,7318938817.36,6038168,72897450,153732,18317632.3,22653584.93333334,24695687.58888889,1.521480,1.802997,1.456598,406.050000,36.87356987,Technology Services,1.33390343,0.9529795,1.0291758,0.8906199,0.006283,0.000648,0.102547,0.069518,0.511495,1.35116077,0.72525565,0.29577401,0.69349908,0.17078787,420.820,USD,420.82,USD,420.82,USD,420.82,USD,420.82000,USD,407.7700,USD,407.7840,USD,407.7840,USD,408.2100,USD,408.2100,USD,408.2100,USD,408.2100,USD,417.35,USD,417.35,USD,958891.40271493,USD,407.44680,407.458012,407.66144,406.709715,408.077949,407.557865,407.037781,408.834448,405.770310,402.706172,Titans
1,NVDA,NVIDIA Corporation,NASDAQ,2.294911e+12,USD,917.9644,USD,889.17,USD,14.55,USD,1.640361,0.244645,2399492500,24983933,196861538,963981,50731960.4,52039927.09999999,45684760.52222235,3.669561,3.465403,3.660522,911.288167,76.90398358,Electronic Technology,1.55029011,1.8023118,2.3419523,1.7245569,0.003040,0.000376,0.169068,0.108818,0.714603,1.8226155,0.93532542,0.43920408,0.87974948,0.24321959,919.880,USD,919.88,USD,919.88,USD,919.88,USD,919.88000,USD,917.7846,USD,918.6000,USD,918.6000,USD,919.8800,USD,919.8800,USD,919.8800,USD,919.8800,USD,919.88,USD,919.88,USD,2058175.67567568,USD,917.04892,917.041488,917.50708,914.952654,919.480428,917.509580,915.538732,922.645829,911.518965,900.392101,Titans
2,AMZN,"Amazon.com, Inc.",NASDAQ,1.826434e+12,USD,175.8320,USD,172.50,USD,1.32,USD,0.760763,-0.582099,9227833763.2,10147300,116848089,216491,36604066.00000006,47647839.56666667,48387894.11111113,1.818962,1.727769,1.783302,175.407300,60.84994463,Retail Trade,1.18742275,1.1343786,1.3817025,1.1703976,0.002072,0.000255,0.231699,0.153234,0.740425,1.57087472,0.68843918,0.24672143,0.54233525,0.13228522,180.140,USD,180.14,USD,180.14,USD,180.14,USD,188.65391,USD,175.8100,USD,175.9699,USD,175.9699,USD,176.6699,USD,176.6699,USD,176.6699,USD,176.6699,USD,180.14,USD,180.14,USD,376908.19672131,USD,175.75702,175.806900,176.03200,175.929615,176.618810,176.080100,175.541390,177.031126,175.433745,173.836364,Titans
3,GOOGL,Alphabet Inc.,NASDAQ,1.667862e

In [377]:
# drop NaNs
tv_df = trading_view_raw_df.dropna()
print(f'NaNs = {tv_df.isna().sum().sum()}')

# confirm no NaNs
if tv_df.isna().sum().sum() == 0:
    nan_count = tv_df.isna().sum().sum()
    print(f"There are zero {nan_count} NaNs in DataFrame! Nice work!")
else:
    print(f"There are {tv_df.isna().sum().sum()} NaNs in DataFrame, check your code dumb dumb!")

NaNs = 0
There are zero 0 NaNs in DataFrame! Nice work!


In [378]:
# view the raw counts and percentages of each market cap type
market_cap_type_df = trading_view_raw_df['marketCapType'].value_counts().reset_index()
market_cap_type_df.columns = ['marketCapType', 'count']  # Rename columns for clarity
# calculate percentage -> add new columne to dataframe
market_cap_type_df['percentage'] = (market_cap_type_df['count'] / market_cap_type_df['count'].sum()) * 100

print(f'There are still the same {len(trading_view_raw_df.index)} U.S. Stocks in the dataframe \n$(pre-market change) > $0.00 USD')
market_cap_type_df

There are still the same 1645 U.S. Stocks in the dataframe 
$(pre-market change) > $0.00 USD


,marketCapType,count,percentage
0,Small caps,377,22.917933
1,Large caps,370,22.492401
2,Midlers,316,19.209726
3,Shrimp,277,16.838906
4,Micro caps,250,15.197568
5,Titans,40,2.431611
6,Undefined,15,0.911854


In [379]:
# print length of df
print(f'{len(tv_df.index)} U.S. Stocks \n$(pre-market change) > $0.00 USD')
# print sample head
tv_df.sample(5)

1630 U.S. Stocks 
$(pre-market change) > $0.00 USD


,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
154,COF,Capital One Financial Corporation,NYSE,5.211569e+10,USD,137.07,USD,136.5000,USD,0.7600,USD,0.559894,0.559894,375100049.66,341760,8427601,175,2566624.2,2972967.2,2806314.5,1.789512,2.022737,2.146076,137.200000,11.47067684,Finance,0.6480764,1.3807292,1.0957123,1.4781015,0.169380,0.042718,0.385377,0.220984,0.305340,0.56812187,0.3003873,0.11519165,0.65468189,0.1881108,139.9500,USD,140.860,USD,140.8600,USD,140.8600,USD,177.950000,USD,137.052,USD,137.19,USD,137.19,USD,137.19,USD,138.1800,USD,138.1800,USD,138.1800,USD,139.9500,USD,139.9500,USD,945621.02063978,USD,137.083400,137.030875,136.975400,136.931692,137.132122,136.973285,136.814448,137.688862,137.102625,136.516388,Large caps
1453,SPI,"SPI Energy Co., Ltd.",NASDAQ,2.054202e+07,USD,0.65,USD,0.6311,USD,0.0131,USD,2.119741,2.119741,23219018.7917,5326,151264,468,54791.1,81535.46666667,90319.13333333,5.063291,5.702449,8.123080,0.644000,-,Electronic Technology,-,1.8635943,2.3093488,0.71298164,0.277001,0.239192,0.099289,0.057113,0.355016,0.36844695,0.20401059,0.08648968,0.42634329,0.11108534,0.7575,USD,1.260,USD,1.2600,USD,1.6100,USD,1890.000000,USD,0.650,USD,0.65,USD,0.65,USD,0.65,USD,0.6500,USD,0.6500,USD,0.6500,USD,0.6758,USD,0.6758,USD,561765.82278551,USD,0.639588,0.640980,0.641548,0.638361,0.650951,0.635507,0.620063,0.652844,0.633967,0.615090,Shrimp
1204,DNMR,"Danimer Scientific, Inc.",NYSE,1.663175e+08,USD,1.63,USD,1.5600,USD,0.1000,USD,6.410256,0.000000,89510713.92500001,457478,3374078,71096,768950.4,929128.56666667,1213030.07777778,8.974359,11.467092,13.736762,1.630000,-,Process Industries,2.3317344,2.7334404,1.7220556,1.5456183,0.110963,0.023347,0.148484,0.084697,0.534880,1.85304498,1.01687031,0.56636922,0.66395935,0.182911,1.7000,USD,1.700,USD,2.0900,USD,4.5900,USD,66.299900,USD,1.630,USD,1.63,USD,1.63,USD,1.63,USD,1.7000,USD,1.7000,USD,1.7000,USD,1.7000,USD,1.7000,USD,196376.38376384,USD,1.627200,1.618875,1.609200,1.593669,1.634900,1.603455,1.572010,1.638637,1.599545,1.560453,Micro caps
1401,LSDI,Lucy Scientific Discovery Inc.,NASDAQ,3.388086e+07,USD,1.92,USD,1.8500,USD,0.1882,USD,10.941860,7.558140,11132218.770488,81472,162162,3658,41978.7,31315.53333333,8482

In [380]:
# drop undefined
tv_df = tv_df[tv_df['marketCapType'] != 'Undefined']

market_cap_type_df2 = tv_df['marketCapType'].value_counts().reset_index()
market_cap_type_df2.columns = ['marketCapType', 'count']  # Rename columns for clarity

# calculate percentage for each market cap type and add it to the DataFrame
market_cap_type_df2['percentage'] = (market_cap_type_df2['count'] / market_cap_type_df2['count'].sum()) * 100

market_cap_type_df2
print(market_cap_type_df2)


  marketCapType  count  percentage
0    Small caps    377   23.128834
1    Large caps    370   22.699387
2       Midlers    316   19.386503
3        Shrimp    277   16.993865
4    Micro caps    250   15.337423
5        Titans     40    2.453988


In [381]:
# 1/3 #### CONFIG DATA SECTION ####
criteria_config = {
    "Titans": {
        "pre_market_change_pct_threshold": 0.002,  # Reduced to 0.2% for Titans
        "float_shares_outstanding_threshold": 1000000000,  # Increased to 1 billion shares
        "relative_volume_threshold": 1.2,  # Slightly more inclusive
        "relative_volume_at_time_threshold": 0.03,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.001,  # Reduced to 0.1%
        "pre_market_vmap_drawdown_threshold": 0.003  # .3% drawdown from VWAP
    },
    "Large caps": {
        "pre_market_change_pct_threshold": 0.005,  # Reduced to 0.5% for Large caps
        "float_shares_outstanding_threshold": 200000000,  # Increased to 200 million shares
        "relative_volume_threshold": 1.3,  # Slightly more inclusive
        "relative_volume_at_time_threshold": 0.04,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.005,  # Reduced to 0.5%
        "pre_market_vmap_drawdown_threshold": 0.004  # .4% drawdown from VWAP
    },
    "Midlers": {
        "pre_market_change_pct_threshold": 0.02,  # 2% for Mid caps
        "float_shares_outstanding_threshold": 50000000,  # 50 million shares
        "relative_volume_threshold": 1.3,
        "relative_volume_at_time_threshold": .05,
        "pre_market_gap_percentage_threshold": .02,
        "pre_market_vmap_drawdown_threshold": 0.005  # .5% drawdown from VWAP
    },
    "Small caps": {
        "pre_market_change_pct_threshold": 0.03,  # 3% for Small caps
        "float_shares_outstanding_threshold": 20000000,  # 20 million shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": .05,
        "pre_market_gap_percentage_threshold": .03,
        "pre_market_vmap_drawdown_threshold": 0.006  # 6% drawdown from VWAP
    },
    "Micro caps": {
        "pre_market_change_pct_threshold": 0.04,  # 4% for Micro caps
        "float_shares_outstanding_threshold": 5000000,  # 5 million shares
        "relative_volume_threshold": 1.1,
        "relative_volume_at_time_threshold": .05,
        "pre_market_gap_percentage_threshold": .04,
        "pre_market_vmap_drawdown_threshold": 0.007  # 7% drawdown from VWAP
    },
    "Shrimp": {
        "pre_market_change_pct_threshold": 0.05,  # 5% for Shrimp
        "float_shares_outstanding_threshold": 1000000,  # 1 million shares
        "relative_volume_threshold": 1.0,
        "relative_volume_at_time_threshold": .05,
        "pre_market_gap_percentage_threshold": .05,
        "pre_market_vmap_drawdown_threshold": 0.008  # 8% drawdown from VWAP
    }
}

# 2/3 #### CORE FUNCTIONS ####
def filter_by_pre_market_change(df, change_pct_threshold):
    """Filter stocks by pre-market change percentage."""
    return df[df['Pre-market Change %'] >= change_pct_threshold]

def filter_by_float_shares(df, float_shares_threshold):
    """Filter stocks by float shares outstanding."""
    # create a copy to safely modify without affecting the original DataFrame
    modified_df = df.copy()
    modified_df['Float shares outstanding'] = pd.to_numeric(modified_df['Float shares outstanding'], errors='coerce')
    return modified_df[modified_df['Float shares outstanding'] <= float_shares_threshold]

def filter_by_relative_volume(df, relative_volume_threshold):
    """Filter stocks by relative volume."""
    df['Relative Volume 1 day'] = pd.to_numeric(df['Relative Volume 1 day'], errors='coerce')
    return df[df['Relative Volume 1 day'] >= relative_volume_threshold]

def filter_by_relative_volume_at_time(df, relative_volume_at_time_threshold):
    """Filter stocks by relative volume at a specific time."""
    # Convert 'Relative Volume at Time' to float
    df['Relative Volume at Time'] = pd.to_numeric(df['Relative Volume at Time'], errors='coerce')
    return df[df['Relative Volume at Time'] >= relative_volume_at_time_threshold]

def filter_by_pre_market_gap_percentage(df, pre_market_gap_percentage_threshold):
    """Filter stocks by pre-market gap percentage."""
    return df[df['Pre-market Gap %'] >= pre_market_gap_percentage_threshold]

def filter_by_price_near_vwap(df, pre_market_vmap_drawdown_threshold):
    """
    Filter stocks where the price is near the VWAP, considering the specified drawdown threshold.
    """
    df_filtered = df.copy()
    df_filtered['Price'] = pd.to_numeric(df_filtered['Price'], errors='coerce')
    df_filtered['Volume Weighted Average Price 1 day'] = pd.to_numeric(df_filtered['Volume Weighted Average Price 1 day'], errors='coerce')
    
    # Calculate the acceptable minimum price based on the VWAP drawdown threshold
    df_filtered['Min Price from VWAP'] = df_filtered['Volume Weighted Average Price 1 day'] * (1 - pre_market_vmap_drawdown_threshold)
    
    # Apply the filter
    df_filtered = df_filtered[df_filtered['Price'] >= df_filtered['Min Price from VWAP']]
    
    return df_filtered.drop(columns=['Min Price from VWAP'])

"""SPECIAL FILTERS"""
def filter_by_volatility(df):
    """Filter stocks based on recent volatility being higher than weekly and monthly averages."""
    # Ensure data types are correct; convert to numeric if necessary
    df['Volatility 1 day'] = pd.to_numeric(df['Volatility 1 day'], errors='coerce')
    df['Volatility 1 week'] = pd.to_numeric(df['Volatility 1 week'], errors='coerce')
    df['Volatility 1 month'] = pd.to_numeric(df['Volatility 1 month'], errors='coerce')
    filtered_df = df[
        (df['Volatility 1 day'] >= df['Volatility 1 week']) & 
        (df['Volatility 1 day'] >= df['Volatility 1 month'])]
    return filtered_df

# 3/3 #### WRAPPER FUNCTION ####
def screen_stocks_by_category(df, category):
    """Screen stocks based on their market cap category."""
    # 3a. Get the configuration for the specified category
    config = criteria_config.get(category, {})
    pre_market_change_pct_threshold = config.get("pre_market_change_pct_threshold", 0)
    float_shares_threshold = config.get("float_shares_outstanding_threshold", float('inf'))
    relative_volume_threshold = config.get("relative_volume_threshold", 0)
    relative_volume_at_time_threshold = config.get("relative_volume_at_time_threshold", 0)
    pre_market_gap_percentage_threshold = config.get("pre_market_gap_percentage_threshold", 0)
    pre_market_vmap_drawdown_threshold = config.get("pre_market_vmap_drawdown_threshold", 0)
    # 3b. Apply the filters
    filtered_df = filter_by_pre_market_change(df, pre_market_change_pct_threshold)
    filtered_df = filter_by_float_shares(filtered_df, float_shares_threshold)
    filtered_df = filter_by_relative_volume(filtered_df, relative_volume_threshold)
    filtered_df = filter_by_relative_volume_at_time(filtered_df, relative_volume_at_time_threshold)
    filtered_df = filter_by_volatility(filtered_df) # SPECIAL FILTER
    # filtered_df = filter_by_price_above_vwap(filtered_df) # SPECIAL FILTER
    filtered_df = filter_by_pre_market_gap_percentage(filtered_df, pre_market_gap_percentage_threshold)
    filtered_df = filter_by_price_near_vwap(filtered_df, pre_market_vmap_drawdown_threshold)
  
    # 3c. Return the filtered DataFrame
    return filtered_df

In [382]:
#EXECUTE
tv_L2_df = pd.DataFrame()
categories = tv_df['marketCapType'].unique()

for category in categories:
    category_df = tv_df[tv_df['marketCapType'] == category]
    screened_df = screen_stocks_by_category(category_df, category)
    tv_L2_df = pd.concat([tv_L2_df, screened_df])

# Display results
print(f'Original DataFrame length: {len(tv_df.index)}')
print(f'Number of rows removed: {len(tv_df.index) - len(tv_L2_df.index)}')
print(f'NEW DataFrame length: {len(tv_L2_df.index)}')

# filter in descending order by market cap and then by pre-market change percentage
tv_L2_df = tv_L2_df.sort_values(by=['Market capitalization', 'Pre-market Change %'], ascending=[False, False])
# reset index and drop
tv_L2_df = tv_L2_df.reset_index(drop=True)

Original DataFrame length: 1630
Number of rows removed: 1626
NEW DataFrame length: 4


In [383]:
# create a breakdown of the market cap types
market_cap_type_df3 = tv_L2_df['marketCapType'].value_counts().reset_index()
market_cap_type_df3.columns = ['marketCapType', 'count']  # Rename columns for clarity
# calculate percentage for each market cap type and add it to the DataFrame
market_cap_type_df3['percentage'] = (market_cap_type_df3['count'] / market_cap_type_df3['count'].sum()) * 100

print('Returned Gap-Up Market Cap Type Groups:')
market_cap_type_df3

Returned Gap-Up Market Cap Type Groups:


,marketCapType,count,percentage
0,Large caps,1,25.0
1,Midlers,1,25.0
2,Small caps,1,25.0
3,Shrimp,1,25.0


In [384]:
# market cap type groups
print(f"Market Cap Groups:\n{tv_L2_df['marketCapType'].unique()}\n")

# TradingView column names
tv_raw_columns_list = tv_L2_df.columns.tolist()
print(f'TradingView Column Names (raw data):\n{tv_raw_columns_list}\n')

# stocks returned from the pre-market gap up screener
print(f'{len(tv_L2_df.index)} = U.S. stocks returned from the pre-market gap up screener.')

# create a list of 'Symbols' from the tv_L2_df
gap_up_stocks_list = tv_L2_df['Symbol']
gap_up_stocks_list = gap_up_stocks_list.tolist()
print(gap_up_stocks_list)

# FILTERD DATAFRAME WITH GAP-UP STOCK TARGETS
tv_L2_df


Market Cap Groups:
['Large caps' 'Midlers' 'Small caps' 'Shrimp']

TradingView Column Names (raw data):
['Symbol', 'Description', 'Exchange', 'Market capitalization', 'Market capitalization - Currency', 'Price', 'Price - Currency', 'Pre-market Open', 'Pre-market Open - Currency', 'Pre-market Change', 'Pre-market Change - Currency', 'Pre-market Change %', 'Pre-market Gap %', 'Float shares outstanding', 'Volume 1 day', 'Volume 1 week', 'Pre-market Volume', 'Average Volume 10 days', 'Average Volume 30 days', 'Average Volume 90 days', 'Volatility 1 day', 'Volatility 1 week', 'Volatility 1 month', 'Volume Weighted Average Price 1 day', 'Price to earnings ratio', 'Sector', 'Relative Volume at Time', 'Beta 1 year', 'Beta 3 years', 'Beta 5 years', 'Relative Volume 1 minute', 'Relative Volume 5 minutes', 'Relative Volume 15 minutes', 'Relative Volume 30 minutes', 'Relative Volume 1 hour', 'Relative Volume 2 hours', 'Relative Volume 4 hours', 'Relative Volume 1 day', 'Relative Volume 1 week', 'R

,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
0,AGR,"Avangrid, Inc.",NYSE,1.398403e+10,USD,36.1550,USD,33.60,USD,3.72,USD,11.596010,4.738155,6.853858e+07,2556937,4898242,162256,890896.7,799781.7,923876.26666667,11.818182,2.209133,2.215244,36.070000,17.80859029,Utilities,44.310116,0.89043146,0.7404226,0.54155004,0.008215,0.001919,0.466285,0.451788,1.195366,19.27370505,11.25233605,3.305672,1.44226988,0.31088383,36.3050,USD,36.3050,USD,36.3050,USD,41.2200,USD,57.240,USD,36.1500,USD,36.1750,USD,36.1750,USD,36.3050,USD,36.3050,USD,36.3050,USD,36.3050,USD,36.3050,USD,36.3050,USD,1040005.00062508,USD,36.15900,36.161250,36.17500,36.169231,36.240054,36.173250,36.106446,36.284194,36.121300,35.958406,Large caps
1,ARHS,"Arhaus, Inc.",NASDAQ,2.100035e+09,USD,14.9700,USD,13.41,USD,1.18,USD,9.204368,4.602184,3.849788e+07,1844451,5509621,18588,886492.2,729934.2,905800.26666667,20.072464,6.089707,3.844914,14.786667,14.85413773,Consumer Durables,21.535353,2.1525056,2.3993912,2.3993912,0.034688,0.008684,0.187786,0.159855,0.742167,6.67563224,4.8590718,2.410472,1.59420442,0.34527383,15.5900,USD,15.5900,USD,15.5900,USD,15.5900,USD,15.590,USD,14.9800,USD,15.1100,USD,15.1100,USD,15.1400,USD,15.5900,USD,15.5900,USD,15.5900,USD,15.5900,USD,15.5900,USD,579683.01886792,USD,15.02900,15.020625,14.95300,14.964377,15.121820,14.952750,14.783680,15.263195,14.929125,14.595055,Midlers
2,ML,MoneyLion Inc.,NYSE,7.195275e+08,USD,69.9800,USD,53.84,USD,9.45,USD,17.982873,2.454805,4.834745e+06,336986,791941,16874,128262.4,113884.2,133646.32222222,31.351256,9.312434,6.593099,66.417467,-,Technology Services,12.848809,2.4364288,3.0196884,2.6366432,0.047272,0.013522,0.312359,0.280489,1.454853,10.71768588,6.43124759,3.300665,1.26885093,0.45998353,70.8623,USD,70.8623,USD,70.8623,USD,70.8623,USD,387.387,USD,70.1300,USD,70.8623,USD,70.8623,USD,70.8623,USD,70.8623,USD,70.8623,USD,70.8623,USD,70.8623,USD,70.8623,USD,503316.10044313,USD,70.15236,70.327675,69.68292,68.237277,71.266676,69.390585,67.514494,72.638218,66.512145,60.386072,Small caps
3,LYT,Lytus Technologies Holdings PTV. Ltd.,NASDAQ,3.606651e+06,USD,13.5074,USD,6.96,USD,7.14,USD,309.090909,201.298701,3.210263e+05,37786097,42444620,5882429,4454003.6,1607600.

| Gap-Up Screener |
|--|
IEX Cloud API (GET Request)

In [385]:
"""
A.
THIS IS ALSO GOING TO BE YOUR NEWS, CURRENT NEWS (like almost real-time (within 15-30mins))
-try to get foreign news where people are awake
-translation (internationalization) feature would be both advanced as well as practical
-all info about these companies so we start to learn what we are wokring with today
-maybe some fundamentals although less important for gap up? unless gap up turns into swing trade

B.
-need a closeness list. some rating system. avoid if a stock barely misses one or two filter metrics.
-so it's two part, 1-the filter closeness boolean as well as well has how close this miss was in terms of percentage and like an aggregate or miss difference -> THEN scoring system?

C.
-you are going to need also a -> 'chart_tv_df (this will be for the TradingView data)

"""

"\nA.\nTHIS IS ALSO GOING TO BE YOUR NEWS, CURRENT NEWS (like almost real-time (within 15-30mins))\n-try to get foreign news where people are awake\n-translation (internationalization) feature would be both advanced as well as practical\n-all info about these companies so we start to learn what we are wokring with today\n-maybe some fundamentals although less important for gap up? unless gap up turns into swing trade\n\nB.\n-need a closeness list. some rating system. avoid if a stock barely misses one or two filter metrics.\n-so it's two part, 1-the filter closeness boolean as well as well has how close this miss was in terms of percentage and like an aggregate or miss difference -> THEN scoring system?\n\nC.\n-you are going to need also a -> 'chart_tv_df (this will be for the TradingView data)\n\n"

In [386]:
"""NEWS AND THAT INTERNATIONALIZATION SHIT GOES HERE"""
info_news_df = pd.DataFrame(tv_L2_df, columns=['Symbol', 'Exchange', 'marketCapType'])

info_news_df

,Symbol,Exchange,marketCapType
0,AGR,NYSE,Large caps
1,ARHS,NASDAQ,Midlers
2,ML,NYSE,Small caps
3,LYT,NASDAQ,Shrimp


In [387]:
# # manual add list
# manual_add_list = ['DELL','PLTR','TSLA']
# # append list with manually added stocks
# gap_up_stocks_list.extend(manual_add_list)

# print(f'{manual_add_list} added manually\n')
# print(f'New list: {gap_up_stocks_list}')

In [388]:
#### IEX CLOUD API KEY -> VARIABLE ####
iex_api_key = os.getenv("IEX_API_KEY")

# set the list of symbols variable
symbols = ','.join(gap_up_stocks_list)

# SETTING THE DATE RANGE
"""
y - Example: range=1y -> returns (one year)
ytd - Example: range=ytd -> returns (year-to-date)
m - Example: range=6m returns (six months)
d - Example: range=5d returns (five days)
"""
data_range = '4m' # DATE RANGE (for historical stock price data)

url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols}&types=chart&range={data_range}&token={iex_api_key}'

# Make the GET request
response = requests.get(url)

# Initialize a list to collect DataFrames for each symbol
dataframes_list = []

# Check if the request was successful
if response.status_code == 200:
    # Convert the response to JSON
    data = response.json()

    # Process and print the data with a progress bar
    for symbol in tqdm(gap_up_stocks_list, desc="Processing symbols"):
        if symbol in data:
            # Create a DataFrame for the current symbol's data
            symbol_df = pd.DataFrame(data[symbol]['chart'])
            # Add a 'Ticker' column to identify the data
            symbol_df['Ticker'] = symbol
            # Append the current symbol's DataFrame to the list
            dataframes_list.append(symbol_df)
            # print(f'Data for {symbol}:')
            # print(symbol_df.head(5))  # print the first few rows of the DataFrame for this symbol
else:
    print(f'Failed to fetch data: {response.status_code}')

# concatenate dataframes and create a combined list
historical_df = pd.concat(dataframes_list, ignore_index=True)


Processing symbols: 100%|██████████| 4/4 [00:00<00:00, 1915.86it/s]


In [389]:
# Set variables for historical data analysis
historical_length = len(historical_df.index)
unique_stocks = historical_df['Ticker'].nunique()
start_date = historical_df['date'].min()
end_date = historical_df['date'].max()

print(f"START date: {start_date}")
print(f"END date: {end_date}")
print(f'{historical_df["date"].nunique()} days of historical stock data.\n')
print(f"{historical_length} = Length of dataframe")
print(f"{unique_stocks} = Unique stocks in dataframe")

# Entries per stock
days_per_stock = historical_length / unique_stocks
print(f"{days_per_stock} = Date entries per stock (MATH check -> should be the same as the unique )\n")

# View the stocks
print(f'Unique stock ticker symbols: {gap_up_stocks_list}')
historical_cols_list = historical_df.columns.tolist()
print(f'Columns: {historical_cols_list}')

# convert df to .csv
historical_df.to_csv(f'historical_stock_data{today_date_str}.csv', index=False)

# Display the first few rows of the dataframe
historical_df.head()

START date: 2023-11-08
END date: 2024-03-07
82 days of historical stock data.

328 = Length of dataframe
4 = Unique stocks in dataframe
82.0 = Date entries per stock (MATH check -> should be the same as the unique )

Unique stock ticker symbols: ['AGR', 'ARHS', 'ML', 'LYT']
Columns: ['close', 'high', 'low', 'open', 'priceDate', 'symbol', 'volume', 'id', 'key', 'subkey', 'date', 'updated', 'changeOverTime', 'marketChangeOverTime', 'uOpen', 'uClose', 'uHigh', 'uLow', 'uVolume', 'fOpen', 'fClose', 'fHigh', 'fLow', 'fVolume', 'label', 'change', 'changePercent', 'Ticker']


,close,high,low,open,priceDate,symbol,volume,id,key,subkey,date,updated,changeOverTime,marketChangeOverTime,uOpen,uClose,uHigh,uLow,uVolume,fOpen,fClose,fHigh,fLow,fVolume,label,change,changePercent,Ticker
0,30.56,30.940,30.18,30.94,2023-11-08,AGR,1172948.0,HISTORICAL_PRICES,AGR,,2023-11-08,1701468983000,0.000000,0.000000,30.94,30.56,30.940,30.18,1172948,30.5056,30.1310,30.5056,29.7563,1172948.0,"Nov 8, 23",0.00,0.0000,AGR
1,30.21,30.830,30.19,30.63,2023-11-09,AGR,672508.0,HISTORICAL_PRICES,AGR,,2023-11-09,1701468985000,-0.011453,-0.011453,30.63,30.21,30.830,30.19,672508,30.2000,29.7859,30.3972,29.7661,672508.0,"Nov 9, 23",-0.35,-0.0115,AGR
2,30.38,30.500,30.09,30.38,2023-11-10,AGR,836532.0,HISTORICAL_PRICES,AGR,,2023-11-10,1701468987000,-0.005890,-0.005890,30.38,30.38,30.500,30.09,836532,29.9535,29.9535,30.0718,29.6675,836532.0,"Nov 10, 23",0.17,0.0056,AGR
3,29.35,30.355,29.33,30.19,2023-11-13,AGR,846073.0,HISTORICAL_PRICES,AGR,,2023-11-13,1701468982000,-0.039594,-0.039594,30.19,29.35,30.355,29.33,846073,29.7661,28.9379,29.9288,28.9182,846073.0,"Nov 13, 23",-1.03,-0.0339,AGR
4,31.40,31.430,30.33,30.33,2023-11-14,AGR,1307865.0,HISTORICAL_PRICES,AGR,,2023-11-14,1701468982000,0.027487,0.027487,30.33,31.40,31.430,30.33,1307865,29.9042,30.9592,30.9887,29.9042,1307865.0,"Nov 14, 23",2.05,0.0698,AGR


In [390]:
#### DEV - DATA ORGANIZATION ####
# convert df to .csv
historical_df.to_csv(f'/Users/sudz4/Desktop/FINANCIAL-SYSTEMS/dark-trader/trading_view_raw_data/historical_stock_data{today_date_str}.csv', index=False)

In [391]:
# Ensure the dataframe is sorted by date if it's not already
historical_sorted_df = historical_df.sort_values(by='date')

# Select mandatory columns for a historical price bar chart + volume
# and explicitly copy the dataframe to ensure it's not a view but a new object
chart_historical_df = historical_sorted_df[[
    'symbol',   # *MANDATORY
    'date',     # *MANDATORY
    'close',    # *MANDATORY
    'high',     # *MANDATORY
    'low',      # *MANDATORY
    'open',     # *MANDATORY
    'volume'    # *MANDATORY
]].copy()

# Group by 'symbol' and get the last occurrence
last_date_df = chart_historical_df.groupby('symbol').last().reset_index()

last_date_df


,symbol,date,close,high,low,open,volume
0,AGR,2024-03-07,36.24,36.3050,35.7500,35.800,5317215.0
1,ARHS,2024-03-07,14.37,15.5900,13.8000,14.000,3614594.0
2,LYT,2024-03-07,8.70,18.6216,8.0500,9.516,75316515.0
3,ML,2024-03-07,67.36,70.8623,58.4101,59.460,764686.0


In [392]:
# re-index the dataframe
chart_historical_df.reset_index(drop=True, inplace=True)
# view head of your main df
print(f'Length of historical data: {len(chart_historical_df.index)}')

Length of historical data: 328


In [393]:
"""
YOUR DATAFRAMES TO MOVE FORWARD WITH:
"""
#### INFO / NEWS DATAFRAME ####
print(f'1 - NEWS & STOCK INFO (trending news etc.):\n{info_news_df}\n')

#### IEX (API) - HISTORICAL DATA FRAME ####
print('2a - IEX HISTORICAL data (START):')
print(f'{chart_historical_df.head()}\n')
print('2b - IEX HISTORICAL data (END):')
print(f'{chart_historical_df.tail()}\n')

1 - NEWS & STOCK INFO (trending news etc.):
  Symbol Exchange marketCapType
0    AGR     NYSE    Large caps
1   ARHS   NASDAQ       Midlers
2     ML     NYSE    Small caps
3    LYT   NASDAQ        Shrimp

2a - IEX HISTORICAL data (START):
  symbol        date    close     high      low     open     volume
0    AGR  2023-11-08  30.5600  30.9400  30.1800  30.9400  1172948.0
1     ML  2023-11-08  26.1900  28.0300  21.8800  21.8800   127887.0
2   ARHS  2023-11-08   8.8600   9.1197   8.8200   9.0000   713824.0
3    LYT  2023-11-08   0.1410   0.1458   0.1339   0.1434   188658.0
4    LYT  2023-11-09   0.1317   0.1450   0.1202   0.1400   443148.0

2b - IEX HISTORICAL data (END):
    symbol        date  close     high      low    open      volume
323    AGR  2024-03-06  32.08  32.3900  31.9200  32.100    659052.0
324   ARHS  2024-03-07  14.37  15.5900  13.8000  14.000   3614594.0
325    AGR  2024-03-07  36.24  36.3050  35.7500  35.800   5317215.0
326     ML  2024-03-07  67.36  70.8623  58.4101 

In [394]:
#### TRADINGVIEW (IMPORTED) - DATAFRAME (TODAY'S DATA) ####
print('3 - TRADINGVIEW (IMPORTED) - DATAFRAME (TODAY):')
print(f'U.S. stocks returned from the pre-market gap up screener = {len(tv_L2_df.index)}')
tv_L2_df

3 - TRADINGVIEW (IMPORTED) - DATAFRAME (TODAY):
U.S. stocks returned from the pre-market gap up screener = 4


,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
0,AGR,"Avangrid, Inc.",NYSE,1.398403e+10,USD,36.1550,USD,33.60,USD,3.72,USD,11.596010,4.738155,6.853858e+07,2556937,4898242,162256,890896.7,799781.7,923876.26666667,11.818182,2.209133,2.215244,36.070000,17.80859029,Utilities,44.310116,0.89043146,0.7404226,0.54155004,0.008215,0.001919,0.466285,0.451788,1.195366,19.27370505,11.25233605,3.305672,1.44226988,0.31088383,36.3050,USD,36.3050,USD,36.3050,USD,41.2200,USD,57.240,USD,36.1500,USD,36.1750,USD,36.1750,USD,36.3050,USD,36.3050,USD,36.3050,USD,36.3050,USD,36.3050,USD,36.3050,USD,1040005.00062508,USD,36.15900,36.161250,36.17500,36.169231,36.240054,36.173250,36.106446,36.284194,36.121300,35.958406,Large caps
1,ARHS,"Arhaus, Inc.",NASDAQ,2.100035e+09,USD,14.9700,USD,13.41,USD,1.18,USD,9.204368,4.602184,3.849788e+07,1844451,5509621,18588,886492.2,729934.2,905800.26666667,20.072464,6.089707,3.844914,14.786667,14.85413773,Consumer Durables,21.535353,2.1525056,2.3993912,2.3993912,0.034688,0.008684,0.187786,0.159855,0.742167,6.67563224,4.8590718,2.410472,1.59420442,0.34527383,15.5900,USD,15.5900,USD,15.5900,USD,15.5900,USD,15.590,USD,14.9800,USD,15.1100,USD,15.1100,USD,15.1400,USD,15.5900,USD,15.5900,USD,15.5900,USD,15.5900,USD,15.5900,USD,579683.01886792,USD,15.02900,15.020625,14.95300,14.964377,15.121820,14.952750,14.783680,15.263195,14.929125,14.595055,Midlers
2,ML,MoneyLion Inc.,NYSE,7.195275e+08,USD,69.9800,USD,53.84,USD,9.45,USD,17.982873,2.454805,4.834745e+06,336986,791941,16874,128262.4,113884.2,133646.32222222,31.351256,9.312434,6.593099,66.417467,-,Technology Services,12.848809,2.4364288,3.0196884,2.6366432,0.047272,0.013522,0.312359,0.280489,1.454853,10.71768588,6.43124759,3.300665,1.26885093,0.45998353,70.8623,USD,70.8623,USD,70.8623,USD,70.8623,USD,387.387,USD,70.1300,USD,70.8623,USD,70.8623,USD,70.8623,USD,70.8623,USD,70.8623,USD,70.8623,USD,70.8623,USD,70.8623,USD,503316.10044313,USD,70.15236,70.327675,69.68292,68.237277,71.266676,69.390585,67.514494,72.638218,66.512145,60.386072,Small caps
3,LYT,Lytus Technologies Holdings PTV. Ltd.,NASDAQ,3.606651e+06,USD,13.5074,USD,6.96,USD,7.14,USD,309.090909,201.298701,3.210263e+05,37786097,42444620,5882429,4454003.6,1607600.

In [395]:
"""PICK UP HERE WHERE YOU LEFT OFF"""
# you need to get the chart dataframe to a place where it has all the data you need in there. 
# it would be more next level if we let it rip into live stock feeds from here. all setup with the correct lines.
# in the charts live feed we could use the tradingview html widget to display the charts.


'PICK UP HERE WHERE YOU LEFT OFF'

| KEY LEVELS (Support & Resistance) |
|-|

In [396]:
"calculate key levels from historical data"
# save historical data to a csv
chart_historical_df.to_csv(f'chart_historical_gpt_data{today_date_str}.csv', index=False)

chart_historical_df.head()

,symbol,date,close,high,low,open,volume
0,AGR,2023-11-08,30.5600,30.9400,30.1800,30.9400,1172948.0
1,ML,2023-11-08,26.1900,28.0300,21.8800,21.8800,127887.0
2,ARHS,2023-11-08,8.8600,9.1197,8.8200,9.0000,713824.0
3,LYT,2023-11-08,0.1410,0.1458,0.1339,0.1434,188658.0
4,LYT,2023-11-09,0.1317,0.1450,0.1202,0.1400,443148.0


In [397]:
"""
IMPORTANT:
for effective plotting and time series analysis, it's crucial that the date column in the df is in the datetime format rather than an object (string). 
"""
chart_historical_df['date'] = pd.to_datetime(chart_historical_df['date'])
chart_historical_df.set_index('date', inplace=True)

# view the multi-indexed dataframe
chart_historical_df.head()

,symbol,close,high,low,open,volume
date,,,,,,
2023-11-08,AGR,30.5600,30.9400,30.1800,30.9400,1172948.0
2023-11-08,ML,26.1900,28.0300,21.8800,21.8800,127887.0
2023-11-08,ARHS,8.8600,9.1197,8.8200,9.0000,713824.0
2023-11-08,LYT,0.1410,0.1458,0.1339,0.1434,188658.0
2023-11-09,LYT,0.1317,0.1450,0.1202,0.1400,443148.0


In [400]:
"""SIMPLE MOVING AVERAGES (SMA)"""
def calculate_sma(dataframe, period):
    return dataframe['close'].rolling(window=period).mean()

chart_historical_df['SMA5'] = calculate_sma(chart_historical_df, 5)
chart_historical_df['SMA10'] = calculate_sma(chart_historical_df, 10)
chart_historical_df['SMA15'] = calculate_sma(chart_historical_df, 15)

chart_historical_df.tail(10)


,symbol,close,high,low,open,volume,SMA5,SMA10,SMA15
date,,,,,,,,,
2024-03-05,AGR,31.77,32.4050,31.6300,32.140,675359.0,23.829,27.8025,25.211667
2024-03-05,ARHS,12.93,13.1400,12.5100,12.690,1245225.0,20.007,23.9095,25.192333
2024-03-06,ML,52.55,52.6600,50.3500,51.110,192143.0,29.992,27.8545,28.518333
2024-03-06,LYT,2.31,2.5000,2.2600,2.370,4434724.0,20.398,24.9725,25.336333
2024-03-06,ARHS,12.82,13.2500,12.6050,13.110,1631813.0,22.476,21.2485,24.115667
2024-03-06,AGR,32.08,32.3900,31.9200,32.100,659052.0,22.538,23.1835,26.047667
2024-03-07,ARHS,14.37,15.5900,13.8000,14.000,3614594.0,22.826,21.4165,23.548333
2024-03-07,AGR,36.24,36.3050,35.7500,35.800,5317215.0,19.564,24.7780,25.091000
2024-03-07,ML,67.36,70.8623,58.4101,59.460,764686.0,32.574,26.4860,27.506333


In [402]:
"""EXPONENTIAL MOVING AVERAGES (EMA)"""
def calculate_ema(dataframe, period):
    return dataframe['close'].ewm(span=period, adjust=False).mean()

# Calculate EMAs for the specified periods
chart_historical_df['EMA5'] = calculate_ema(chart_historical_df, 5)
chart_historical_df['EMA10'] = calculate_ema(chart_historical_df, 10)
chart_historical_df['EMA20'] = calculate_ema(chart_historical_df, 20)
chart_historical_df['EMA50'] = calculate_ema(chart_historical_df, 50)

chart_historical_df.tail(10)


,symbol,close,high,low,open,volume,SMA5,SMA10,SMA15,EMA_5,EMA_10,EMA_20,EMA_50,EMA5,EMA10,EMA20,EMA50
date,,,,,,,,,,,,,,,,,
2024-03-05,AGR,31.77,32.4050,31.6300,32.140,675359.0,23.829,27.8025,25.211667,24.529071,24.925685,24.930584,24.451624,24.529071,24.925685,24.930584,24.451624
2024-03-05,ARHS,12.93,13.1400,12.5100,12.690,1245225.0,20.007,23.9095,25.192333,20.662714,22.744652,23.787672,23.999795,20.662714,22.744652,23.787672,23.999795
2024-03-06,ML,52.55,52.6600,50.3500,51.110,192143.0,29.992,27.8545,28.518333,31.291809,28.163806,26.526941,25.119411,31.291809,28.163806,26.526941,25.119411
2024-03-06,LYT,2.31,2.5000,2.2600,2.370,4434724.0,20.398,24.9725,25.336333,21.631206,23.463114,24.220566,24.224924,21.631206,23.463114,24.220566,24.224924
2024-03-06,ARHS,12.82,13.2500,12.6050,13.110,1631813.0,22.476,21.2485,24.115667,18.694137,21.528002,23.134797,23.777672,18.694137,21.528002,23.134797,23.777672
2024-03-06,AGR,32.08,32.3900,31.9200,32.100,659052.0,22.538,23.1835,26.047667,23.156092,23.446547,23.986722,24.103254,23.156092,23.446547,23.986722,24.103254
2024-03-07,ARHS,14.37,15.5900,13.8000,14.000,3614594.0,22.826,21.4165,23.548333,20.227394,21.796266,23.070843,23.721558,20.227394,21.796266,23.070843,23.721558
2024-03-07,AGR,36.24,36.3050,35.7500,35.800,5317215.0,19.564,24.7780,25.091000,25.564930,24.422399,24.325049,24.212477,25.564930,24.422399,24.325049,24.212477
2024-03-07,ML,67.36,70.8623,58.4101,59.460,764686.0,32.574,26.4860,27.506333,39.496620,32.229236,28.423615,25.904537,39.496620,32.229236,28.423615,25.904537


In [ ]:
# LINE LEVELS not full calculation dataframe

| CHARTS |
|-|

# DELL FOCUS - Pre-market

In [ ]:
# # $DELL
# dell_df = key_levels_df[key_levels_df['symbol'] == 'DELL'].copy()
# dell_df['date'] = pd.to_datetime(dell_df['date'])
# dell_df.set_index('date', inplace=True)
# dell_df.head(5)

In [ ]:
# import mplfinance as mpf
# # candlestick chart
# # MANDATORY DATA = 'open', 'high', 'low', 'close', 'volume'

# """
# Part 1:    
# """
# # Part 1a: SMAs -> calculate
# dell_df['SMA20'] = dell_df['close'].rolling(window=20, min_periods=1).mean()
# dell_df['SMA50'] = dell_df['close'].rolling(window=50, min_periods=1).mean()
# # dell_df['SMA200'] = dell_df['close'].rolling(window=200, min_periods=1).mean()

# # Part 1b: VWAP -> calculate
# dell_df['VWAP'] = (dell_df['volume'] * (dell_df['high'] + dell_df['low'] + dell_df['close']) / 3).cumsum() / dell_df['volume'].cumsum()

# """   
# Part 2: 
# """
# add2plot = [
#     #### SMAs ####
#     mpf.make_addplot(dell_df['SMA20'], color='#7BAFD4', width=1.0, label='SMA 20'), # UNC blue
#     mpf.make_addplot(dell_df['SMA50'], color='#003087', width=1.0, label='SMA 50'), # DUKE blue
#     # mpf.make_addplot(dell_df['SMA200'], color='#041E42', width=1.0, label='SMA 200'), # HOYAS blue
#     #### VWAP ####
#     mpf.make_addplot(dell_df['VWAP'], color='#B31B1B', width=1.0, label='VWAP') # BIG RED
# ]

# """   
# Part 3: styling not iterative changes related to data (i.e., color, font, etc.)
# """
# # Use a dark theme style and set text color to white
# mpf_style = mpf.make_mpf_style(base_mpf_style='nightclouds', rc={'text.color': 'white'})

# # Plot the candlestick chart with the moving averages
# fig, axes = mpf.plot(dell_df,
#                      type='candle',
#                      addplot=add2plot,
#                      volume=True,
#                     #  style=mpf_style,
#                      style=s, 
#                      # WTF look at docs (it's not erroring out but going to default style just figure out what that is i think)
#                      figratio=(15, 8),
#                      title='$DELL - Historical Data (6 Months)',
#                      tight_layout=True,
#                      returnfig=True)

# # Add a watermark if needed
# fig.text(0.5, 0.5, 'DELL', fontsize=50, color='gray', alpha=0.5, ha='center', va='center', rotation=0, transform=fig.transFigure)

# plt.show()


In [ ]:
# dell_df.head(5)

In [ ]:
""" 
ISSUE: some weird shit with the style param
nbd tho ctfo. 

workaround =  is first successfully run with style=mpf_style, then swap it for style=s (dark mode - but common view like Ortex)  
the blue shit and mpf_style is actually cool but might confuse user (i.e., blue = red and white = green) with the mpf_style
experienced similar shit like this years ago, idunno will get to it.

"""

' \nISSUE: some weird shit with the style param\nnbd tho ctfo. \n\nworkaround =  is first successfully run with style=mpf_style, then swap it for style=s (dark mode - but common view like Ortex)  \nthe blue shit and mpf_style is actually cool but might confuse user (i.e., blue = red and white = green) with the mpf_style\nexperienced similar shit like this years ago, idunno will get to it.\n\n'

In [ ]:
# you're going to need different charts, one for the pre-market, and at least another for market open
# I'd like market opens to be a live feed but really we might already be in trading view watching the pine script execute

# Trend lines

| END OF IN PROGRESS WORK - NOTES BELOW |
|--|

In [ ]:
# but first start with getting lines on that chart with STRV
# let's get those key levels with an output on them and figure out your strategy
# figure out when fibonacci extensions get activated
# i see one view with a grid layout of the stock charts above or below them the summary with the key levels, current price and all that
# i also see a view where when looking at individiual stocks we got data all over that shit. idgaf if it's "confusing for the user"

# you have your key_levels_df
# so start next from there okay

#### </b> ####

# make title more symetrically bigger not just bigger for fucks sake 
# move price to the right side, keep vol on left
# see if you can get a marker or military like custom google font one of the ones you like
# better yet use that program you wrote for sn logos actually to create your shit
# could put that Sans Peur Systems somewhere at least watermark it, maybe?
# key levels find up to 6?, scoring strategy?, then base allotment of position (1/3, 1/5) conditionally?
# we could add RSI down there, fuck could add ichimoku, do whatever with all this data access now

NOTES -> SMA Legend

In [ ]:
# # SMA20, SMA50, and SMA200 has to be in 'dell_df'
# legend_info = {
#     'Indicator': ['SMA20', 'SMA50', 'SMA200'],
#     'Color': ['blue', 'orange', 'green'],
#     'Description': ['20-day Moving Average', '50-day Moving Average', '200-day Moving Average']
# }

# legend_df = pd.DataFrame(legend_info)
# print(legend_df)


NOTES -> custom mplfinance chart setup example

In [ ]:
# """    
# Custom Theme setup
# """

# dark_style = mpf.make_mpf_style(base_mpf_style='charles', 
#                                 rc={'axes.facecolor': 'black',
#                                     'figure.facecolor': 'darkslategrey',
#                                     'axes.grid': True,
#                                     'grid.color': 'gray',
#                                     'grid.linestyle': '--',
#                                     'grid.linewidth': 0.6,
#                                     'axes.titlesize': 16,
#                                     'axes.titleweight': 'bold',
#                                     'axes.labelsize': 12,
#                                     'axes.labelweight': 'bold',
#                                     'axes.labelcolor': 'white',
#                                     'axes.edgecolor': 'white',
#                                     'axes.linewidth': 2,
#                                     'xtick.color': 'white',
#                                     'xtick.labelsize': 10,
#                                     'ytick.color': 'white',
#                                     'ytick.labelsize': 10,
#                                     'figure.titlesize': 18,
#                                     'figure.titleweight': 'bold',
#                                     'legend.facecolor': 'grey',
#                                     'legend.edgecolor': 'white',
#                                     'legend.fontsize': 10,
#                                     'legend.title_fontsize': 12})

# mpf.plot(strv_df, 
#          type='candle', 
#          style=dark_style,  # Using the custom dark style
#          volume=True, 
#          title='STRV Candlestick Chart',
#          mav=(20, 50, 200),  # Adding moving averages of 20, 50, and 200 periods
#          figratio=(15, 8), 
#          tight_layout=True)

In [ ]:
# i need a math checker for trading days vs calendar days (252/365)

Notes -> yahoo finance API exaple (free) *with limitations

In [ ]:
# import yfinance as yf
# from datetime import datetime, timedelta

In [ ]:
# # yahoo finance data download
# start = datetime.now() - timedelta(days=60)
# end = datetime.now()
# yahoo_df = yf.download(gap_up_stocks_list, start, end)
# yahoo_df = yahoo_df.loc[:,'Close']
# # view the yahoo data yahoo finance formatting multi-index (has a 60day max with standard API maneuvers)
# yahoo_df.head(10)